In [1]:
import csv
import numpy as np
import math

def get_column_average(arr):
    summ = 0
    for i in range(len(arr)):
        summ = summ + float(arr[i][0][2])
    if len(arr) == 0:
        return 0
    else:   
        return summ / len(arr)
    
def find_common_films(u, v, rates):
    u_films_id = []
    v_films_id = []
    indeces_of_u =list(zip(*np.where(rates[:,0]==str(u)))) 
    indeces_of_v =list(zip(*np.where(rates[:,0]==str(v)))) 
    
    for i in range(len(indeces_of_u)):
        u_films_id.append(rates[indeces_of_u[i][0]][1])
    for i in range(len(indeces_of_v)):
        v_films_id.append(rates[indeces_of_v[i][0]][1])
    list_of_common_films =  list(set(u_films_id).intersection(v_films_id))    
    
    return list_of_common_films

def read_file():
    ratings = []
    movies = []
    
    with open("./ratings.csv", 'r', encoding="utf-8") as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            ratings.append(row)
    with open("./movies.csv", 'rt', encoding="utf-8") as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            movies.append(row)
            
    x = list(ratings)
    y = list(movies)
    rates = np.array(x)
    movs = np.array(y)
    
    return [rates,movs]

def print_N_movies(N, rates, movs):
    user_id = 1
    print(movs[np.random.choice(len(movs), size=N, replace=False)])

def print_user_info(user_id, rates, movs):
    indeces_of_user =list(zip(*np.where(rates[:,0]==str(user_id)))) 
    movies_of_user = []
    movie_info = []
    user_movie_rating = []

    for i in range(len(indeces_of_user)):
        movies_of_user.append(rates[indeces_of_user[i][0]][1])
    print("Movie Name       Genre         User Rating")
    for i in range(len(movies_of_user)):
        movie_info.append(movs[np.where(movs[:,0]== movies_of_user[i])] )
        user_movie_rating.append(rates[indeces_of_user[i][0]][2])
        all_info = movie_info[i][0][0] + " " + movie_info[i][0][1] + " " +  movie_info[i][0][2] + " " + rates[indeces_of_user[i][0]][2]
        print(all_info)

def pearson_coefficient_correlation(u, v, ratings, movies):
    u_common_film_ratings = []
    v_common_film_ratings = []
    y_u = 0
    y_v = 0
    list_of_common_films =  find_common_films(u, v, ratings)
    
    for i in range (len(list_of_common_films)):
        u_common_film_ratings.append(ratings[np.where(np.logical_and(ratings[:,0] == str(u), ratings[:,1] == list_of_common_films[i]))])
        v_common_film_ratings.append(ratings[np.where(np.logical_and(ratings[:,0] == str(v), ratings[:,1] == list_of_common_films[i]))])


    y_u = get_column_average(u_common_film_ratings)
    y_v = get_column_average(v_common_film_ratings)
    

    
    upper_value = 0
    bottom_value_1 = 0
    bottom_value_2 = 0
    
    for i in range(len(list_of_common_films)):
        multiplication = (float(u_common_film_ratings[i][0][2]) - float(y_u)) * (float(v_common_film_ratings[i][0][2]) - float(y_v))
        upper_value = upper_value + multiplication
        
        multiplication = pow((float(u_common_film_ratings[i][0][2]) - float(y_u)),2)
        bottom_value_1 = bottom_value_1 + multiplication
        
        multiplication = pow((float(v_common_film_ratings[i][0][2]) - float(y_v)),2)
        bottom_value_2 = bottom_value_2 + multiplication
        

    if (math.sqrt(bottom_value_1) * math.sqrt(bottom_value_2)) == 0:
            
            return -99
    else:
        p_uv = upper_value / (math.sqrt(bottom_value_1) * math.sqrt(bottom_value_2))
        #print("Pearson between " + str(u) + " and " + str(v) + ": " + str(p_uv))
        return p_uv
    
def knn_algorithm(u, k, ratings,movies, array_of_users, prediction, movie):
    np.set_printoptions(suppress=True)
    num_of_users = int(ratings[len(ratings) - 1][0])
    if prediction == False:
        k_neighbors = np.zeros((k,2))
        p_uv = np.zeros((int(ratings[len(ratings) - 1][0]), 2))

        for i in range(1, num_of_users) :
            if i != u:
                p_uv[i- 1][1] = (pearson_coefficient_correlation(u,i,ratings,movies))
                p_uv[i - 1][0] = i

        sorted_p_uv = p_uv[p_uv[:, 1].argsort()]
        for i in range(0, k):
            k_neighbors[i][0] = sorted_p_uv[len(p_uv) - 1 - i][0]
            k_neighbors[i][1] = sorted_p_uv[len(p_uv) - 1 - i][1]

        return k_neighbors
    else:
        p_uv = np.zeros((len(array_of_users), 3))
        for i in range(0,len(array_of_users) ):
            p_uv[i][2] = ratings[np.where(np.logical_and(ratings[:,0] == str(array_of_users[i]), ratings[:,1] == str(movie)))][0][2]
            p_uv[i][1] = pearson_coefficient_correlation(u,array_of_users[i],ratings,movies)
            p_uv[i][0] = array_of_users[i]
        sorted_p_uv = p_uv[p_uv[:, 1].argsort()]
        row_mask = np.apply_along_axis(np.any, 1, sorted_p_uv == -99)
        sorted_p_uv = sorted_p_uv[~row_mask]
        
        row_mask = np.apply_along_axis(np.any, 1, sorted_p_uv < 0)
        sorted_p_uv = sorted_p_uv[~row_mask]
        if len(sorted_p_uv) < k:
            k = len(sorted_p_uv)
        k_neighbors = np.zeros((k,3))

        for i in range(0, k):
            k_neighbors[i][0] = sorted_p_uv[len(sorted_p_uv) - 1 - i][0]
            k_neighbors[i][1] = sorted_p_uv[len(sorted_p_uv) - 1 - i][1]
            k_neighbors[i][2] = sorted_p_uv[len(sorted_p_uv) - 1 - i][2]
        return k_neighbors

def get_average_rating(user,rating, movies):
    indeces_of_user =list(zip(*np.where(rating[:,0]==str(user)))) 
    ratings_of_user = []
    
    for i in range(len(indeces_of_user)):
        ratings_of_user.append(float(rating[indeces_of_user[i][0]][2]))
    #print(ratings_of_user)
    return np.mean(ratings_of_user)

def predict_rating(user, movie, rating, movies, k):
    users_rated_movie = []
    averages_of_neighbors = []
    
    y_u = get_average_rating(user, rating, movies)
    
    users_rated_movie = rating[rating[:, 1] == str(movie), :]
    user_ids = users_rated_movie[:, 0]
    
    neighbors = knn_algorithm(user,k,rating,movies,user_ids, True, movie)
    for i in range(0, len(neighbors)):
        averages_of_neighbors.append(get_average_rating(int(neighbors[i][0]),rating,movies))

    prediction = 0
    upper_value = 0
    bottom_value = 0
    
    for i in range(len(neighbors)):
        upper_value = upper_value + (neighbors[i][1] * (neighbors[i][2] - averages_of_neighbors[i]))
        bottom_value = bottom_value + neighbors[i][1]

    if(bottom_value != 0):
        prediction = y_u + (upper_value / bottom_value)
    else:
        prediction = y_u
    if prediction > 5:
        prediction = 5
    print("Predicted rating: ", prediction)

def get_movies(user, ratings, movies):
    indeces_of_user =list(zip(*np.where(ratings[:,0]==str(user)))) 
    movies_of_user = []
    for i in range(len(indeces_of_user)):
        movies_of_user.append(ratings[indeces_of_user[i][0]][1])
    return movies_of_user


def reccomend_movie(user, rating, movies, k):
    neighbors = knn_algorithm(user, k, rating,movies, 0, False, 0) ##Similar taste neighbors
    movies_of_other_users = []
    movies_of_user = get_movies(int(user), rating, movies)
    for i in range (0, k):
            movies_of_other_users = movies_of_other_users + get_movies(int(neighbors[i][0]), rating, movies)
    
    unwatched_movies = set(movies_of_other_users) - set(movies_of_user)
    unwatched_movies = list(dict.fromkeys(unwatched_movies))
    y_u = get_average_rating(user, rating, movies)
    predictions = np.zeros((len(unwatched_movies),2))
    #print("User average: ", y_u)
    for i in range(len(unwatched_movies)):
        prediction = 0
        users_rated_movie = []
        averages_of_neighbors = []

        users_rated_movie = rating[rating[:, 1] == str(unwatched_movies[i]), :]
        user_ids = users_rated_movie[:, 0]


        if len(user_ids) > k:
            #print("Length condition 1: ", len(user_ids))
            neighbors = knn_algorithm(user,k,rating,movies,user_ids, True, unwatched_movies[i])
            for x in range(0, len(neighbors)):
                averages_of_neighbors.append(get_average_rating(int(neighbors[x][0]),rating,movies))
            
            #print(averages_of_neighbors)
            upper_value = 0
            bottom_value = 0

            for j in range(len(neighbors)):
                upper_value = upper_value + (neighbors[j][1] * (neighbors[j][2] - averages_of_neighbors[j]))
                bottom_value = bottom_value + neighbors[j][1]

            if(bottom_value != 0):
                prediction = y_u + (upper_value / bottom_value)
            else:
                prediction = y_u
            if prediction > 5:
                prediction = 5
            predictions[i][0] = unwatched_movies[i]
            predictions[i][1] = prediction


        elif len(user_ids) < k:

            neighbors = knn_algorithm(user,len(user_ids),rating,movies,user_ids, True, unwatched_movies[i])
            for x in range(0, len(neighbors)):
                averages_of_neighbors.append(get_average_rating(int(neighbors[x][0]),rating,movies))

            upper_value = 0
            bottom_value = 0

            for j in range(len(neighbors)):
                upper_value = upper_value + (neighbors[j][1] * (neighbors[j][2] - averages_of_neighbors[j]))
                bottom_value = bottom_value + neighbors[j][1]

            if(bottom_value != 0):
                prediction = y_u + (upper_value / bottom_value)
            else:
                prediction = y_u
            if prediction > 5:
                prediction = 5
            predictions[i][0] = unwatched_movies[i]
            predictions[i][1] = prediction
            
        elif len(user_ids) == 0:
            prediction = y_u
    predictions = predictions[predictions[:, 1].argsort()]
    print(predictions)
    
ratings, movies = read_file()
print_N_movies(20,ratings,movies)
print_user_info(3,ratings,movies)
#print_user_info(5,ratings,movies)
print("PCC")
print(pearson_coefficient_correlation(6,2,ratings,movies)) ## Primer za 15, 23 ????
#print(pearson_coefficient_correlation(5,187,ratings,movies)) ## Primer za 15, 23 ????

#2915
predict_rating(5,2100,ratings,movies,3)
#predict_rating(5,134130,ratings,movies,3)
reccomend_movie(5,ratings,movies, 2)

[['110390' 'Best Man Down (2012)' 'Drama']
 ['4567' 'American Ninja 3: Blood Hunt (1989)' 'Action|Adventure']
 ['123083' 'Heidi (1993)' 'Children']
 ['77496' 'Night Train to Munich (1941)' 'Film-Noir|Thriller|War']
 ['163931' "Billy Lynn's Long Halftime Walk (2016)" 'Drama|War']
 ['73741' 'Ninja (2009)' 'Action|Crime|Drama|Thriller']
 ['162846' 'The Invisible Maniac (1990)' 'Comedy|Horror|Sci-Fi']
 ['53460' "Surf's Up (2007)" 'Animation|Children|Comedy']
 ['154900' 'Factory (2004)' '(no genres listed)']
 ['157546' 'Adjustment and Work (1986)' 'Documentary']
 ['98509'
  'Bank Error in Your Favour (Erreur de la banque en votre faveur) (2009)'
  'Comedy']
 ['168162'
  'A Perry Mason Mystery: The Case of the Jealous Jokester (1995)'
  '(no genres listed)']
 ['32440' 'If Looks Could Kill (1991)' 'Action|Comedy']
 ['2735' 'Golden Child, The (1986)'
  'Action|Adventure|Comedy|Fantasy|Mystery']
 ['149864' 'Central Airport (1933)' 'Drama']
 ['149836' 'When Harry Tries to Marry (2011)' 'Comedy|R